### Alex Khvatov Homework #3

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
#data = "https://archive.ics.uci.edu/static/public/222/bank+marketing.zip"

In [3]:
#!wget $data
#!unzip ./bank+marketing.zip
#!unzip ./bank.zip
#!rm -rf bank.zip bank+marketing.zip bank.csv bank-names.txt bank-additional.zip

In [4]:
data_file = "bank-full.csv"
df_full = pd.read_csv(data_file, sep=";")

In [5]:
df = df_full[['age', 'job','marital','education', 'balance','housing',
              'contact','day','month','duration','campaign','pdays',
              'previous','poutcome','y']].copy()

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')


df.reset_index()

,index,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,45206,51,technician,married,tertiary,825,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,45207,71,retired,divorced,primary,1729,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,45208,72,retired,married,secondary,5715,no,cellular,17,nov,1127,5,184,3,success,yes
45209,45209,57,blue-collar,married,secondary,668,no,telephone,17,nov,508,4,-1,0,unknown,no


In [6]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


## y variable - has the client subscribed a term deposit or not

In [7]:
#check for missing data
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [8]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [9]:
df.head().T

,0,1,2,3,4
age,58,44,33,47,33
job,management,technician,entrepreneur,blue-collar,unknown
marital,married,single,married,married,single
education,tertiary,secondary,secondary,unknown,unknown
balance,2143,29,2,1506,1
housing,yes,yes,yes,yes,no
contact,unknown,unknown,unknown,unknown,unknown
day,5,5,5,5,5
month,may,may,may,may,may
duration,261,151,76,92,198


In [10]:
df.education.nunique()

4

### Question 1

In [11]:
df.education.mode()

0    secondary
Name: education, dtype: object

### Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

In [12]:
#numerical vs cagegorical columns:
numeric = ['age', 'balance', 'duration', 'day', 'campaign', 'pdays', 'previous']
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [13]:
# df[numeric].corrwith(df[numeric])

In [14]:
# #Correlatoin matrix is 7 by 7 
corr_dict = {}
for col in numeric:
    corr = df[numeric].corrwith(df[col])
    
    corr = corr.drop(col)
    max_corr = corr.idxmax()
    corr_dict[col]=(max_corr, corr[max_corr])
    

In [15]:
max_corr = 0
feature1 = ""
feature2 = ""

for k,v in corr_dict.items():
    column, correlation = v
    if(max_corr<=correlation):
        feature1 = k
        feature2 = column
        max_corr = correlation
print("Answer: Biggest correlation: ", feature1, feature2, max_corr)

Answer: Biggest correlation:  previous pdays 0.4548196354805014


In [16]:
#Another solution from the Zoomcamp QA Bot:
correlation_matrix = df[numeric].corr()
#print(correlation_matrix)
# Get the correlation values and find the maximum
max_corr = correlation_matrix.unstack().sort_values(ascending=False)
# Remove self correlation
max_corr = max_corr[max_corr < 1]
top_features = max_corr.idxmax()
print(f"The two features with the highest correlation are: {top_features}")

The two features with the highest correlation are: ('pdays', 'previous')


### Target encoding

In [17]:
df.y.nunique()

2

In [18]:
#convert to int
df.y = (df.y == 'yes').astype(int)

In [19]:
#global subscription rate
global_subscription_rate = df.y.mean()
print(f"Global subscription rate: {global_subscription_rate}")

Global subscription rate: 0.11698480458295547


### Split the data

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state=42)

In [22]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)


In [23]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [24]:
del df_train['y']
del df_val['y']
del df_test['y']

In [25]:
assert len(df) == len(df_train) + len(df_val) + len(df_test)

### Question 3

* Calculate the mutual information score between __y__ and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using round(score, 2).

In [28]:
from sklearn.metrics import mutual_info_score

In [36]:
def mutual_info_y_score(series):
    return round(mutual_info_score(series, y_train),2)

In [37]:
mi = df_train[categorical].apply(mutual_info_y_score)
mi.sort_values(ascending=False)

month        0.03
poutcome     0.03
job          0.01
housing      0.01
contact      0.01
marital      0.00
education    0.00
dtype: float64

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    * model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [39]:
from sklearn.feature_extraction import DictVectorizer

In [41]:
train_dicts=df_train[categorical + numeric].to_dict(orient='records')

In [48]:
train_dicts[0]

{'job': 'technician',
 'marital': 'single',
 'education': 'tertiary',
 'housing': 'yes',
 'contact': 'cellular',
 'month': 'aug',
 'poutcome': 'unknown',
 'age': 32,
 'balance': 1100,
 'duration': 67,
 'day': 11,
 'campaign': 1,
 'pdays': -1,
 'previous': 0}

In [49]:
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

In [50]:
dv.get_feature_names_out()

array(['age', 'balance', 'campaign', 'contact=cellular',
       'contact=telephone', 'contact=unknown', 'day', 'duration',
       'education=primary', 'education=secondary', 'education=tertiary',
       'education=unknown', 'housing=no', 'housing=yes', 'job=admin.',
       'job=blue-collar', 'job=entrepreneur', 'job=housemaid',
       'job=management', 'job=retired', 'job=self-employed',
       'job=services', 'job=student', 'job=technician', 'job=unemployed',
       'job=unknown', 'marital=divorced', 'marital=married',
       'marital=single', 'month=apr', 'month=aug', 'month=dec',
       'month=feb', 'month=jan', 'month=jul', 'month=jun', 'month=mar',
       'month=may', 'month=nov', 'month=oct', 'month=sep', 'pdays',
       'poutcome=failure', 'poutcome=other', 'poutcome=success',
       'poutcome=unknown', 'previous'], dtype=object)

In [52]:
val_dicts = df_val[categorical + numeric].to_dict(orient='records')

In [53]:
X_val = dv.transform(val_dicts)

In [54]:
from sklearn.linear_model import LogisticRegression

In [55]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [56]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [59]:
y_pred = model.predict_proba(X_val)[:, 1]
y_decision = (y_pred >= 0.5)

In [61]:
accuracy = round((y_val == y_decision).mean(),2)
print("Accuracy of the val

np.float64(0.9)